In [2]:
pip install pushbullet.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 


In [3]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import time
from pushbullet import Pushbullet
import requests
import json
import pandas as pd
import datetime
import pprint
from pytz import timezone
from datetime import datetime

In [5]:
data_path = '/content/drive/MyDrive/Colab Notebooks/20EB 박은서 2022 2학기 프로그래밍 수행평가_날씨 SMS/keyData.json'
with open(data_path, 'r') as file:
  key_data = json.load(file)
  print(type(key_data))

w_key = key_data['key'][0]['weather']
access_token = key_data['key'][0]['sms']

<class 'dict'>


In [6]:
from pandas.io.common import urljoin
from urllib.parse import urlencode, unquote
#날씨 정보
now = datetime.now(timezone('Asia/Seoul'))
day = repr(now.day)
mon = repr(now.month)
year = repr(now.year)
if(now.month<10):mon = '0'+repr(now.month)
if(now.day<10):day = '0'+repr(now.day)
b_d = year + mon + day
print(b_d)

print(year,'년', mon,'월', day, '일', now.hour,'시', now.minute,'분', now.second,'초\n안산시 단원구 와동(57, 122)의 날씨 정보:')


if now.hour<2 or (now.hour==2 and now.minute<=10):
    base_time="0100"
elif now.hour<5 or (now.hour==5 and now.minute<=10): 
    base_time="0200"
elif now.hour<8 or (now.hour==8 and now.minute<=10): 
    base_time="0500"
elif now.hour<=11 or now.minute<=10:
    base_time="0800"
elif now.hour<14 or (now.hour==14 and now.minute<=10): 
    base_time="1100"
elif now.hour<17 or (now.hour==17 and now.minute<=10): 
    base_time="1400"
elif now.hour<20 or (now.hour==20 and now.minute<=10): 
    base_time="1700" 
elif now.hour<23 or (now.hour==23 and now.minute<=10): 
    base_time="2000"
else: 
    base_time="2300"

print(type(base_time))
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
queryParams = {"ServiceKey" : w_key,
                "base_date" : b_d, 
                "base_time" : base_time,
                "nx" : 57,
                "ny" : 122,
                "numOfRows" : "60",
                "pageNo" : "1",
                "dataType" : "JSON"
        }

res = requests.get(url, params = queryParams)
print(res.content)
items = res.json().get('response').get('body').get('items')

contents = res.text
pp = pprint.PrettyPrinter(indent=4)
print(pp.pprint(contents))

data = dict()
data['date'] = b_d

json_ob = json.loads(contents)
print(json_ob)
print(type(json_ob))
body = json_ob['response']['body']['items']
print(body)

weather_data = dict()
for item in items['item']:
  if item['category'] == 'T3H':
    weather_data['tmp'] = item['obsrValue']

  if item['category'] == 'PTY':
        weather_code = item['obsrValue']
        
        if weather_code == '1':
            weather_state = '현재 비가 오고 있습니다. 우산을 챙기세요.'
        elif weather_code == '2':
            weather_state = '현재 비/눈이 오고 있습니다. 우산을 챙기세요.'
        elif weather_code == '3':
            weather_state = '현재 눈이 오고 있습니다. 우산을 챙기고 옷을 따뜻하게 입으세요.'
        elif weather_code == '4':
            weather_state = '현재 소나기가 오고 있습니다. 우산을 챙기세요.'
        elif weather_code == '0':
            weather_state = '현재 비/눈/소나기가 오고 있지 않습니다.'
        else:
          weather_state = '현재 비 혹은 눈이 소량 오고있습니다.'
        
        weather_data['code'] = weather_code
        weather_data['state'] = weather_state





20221006
2022 년 10 월 06 일 17 시 49 분 32 초
안산시 단원구 와동(57, 122)의 날씨 정보:
<class 'str'>
b'{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL_SERVICE"},"body":{"dataType":"JSON","items":{"item":[{"baseDate":"20221006","baseTime":"1700","category":"PTY","nx":57,"ny":122,"obsrValue":"0"},{"baseDate":"20221006","baseTime":"1700","category":"REH","nx":57,"ny":122,"obsrValue":"75"},{"baseDate":"20221006","baseTime":"1700","category":"RN1","nx":57,"ny":122,"obsrValue":"0"},{"baseDate":"20221006","baseTime":"1700","category":"T1H","nx":57,"ny":122,"obsrValue":"16.9"},{"baseDate":"20221006","baseTime":"1700","category":"UUU","nx":57,"ny":122,"obsrValue":"0.6"},{"baseDate":"20221006","baseTime":"1700","category":"VEC","nx":57,"ny":122,"obsrValue":"297"},{"baseDate":"20221006","baseTime":"1700","category":"VVV","nx":57,"ny":122,"obsrValue":"-0.2"},{"baseDate":"20221006","baseTime":"1700","category":"WSD","nx":57,"ny":122,"obsrValue":"0.7"}]},"pageNo":1,"numOfRows":60,"totalCount":8}}}'
'{"re

In [7]:
data['weather'] = weather_data

print(f'[{now}]', '\n안산시 단원구 와동(57, 122)의 날씨 데이터입니다.')
print(f"기온은 {body['item'][3]['obsrValue']}도 입니다.")
print(weather_state)

[2022-10-06 17:49:32.096154+09:00] 
안산시 단원구 와동(57, 122)의 날씨 데이터입니다.
기온은 16.9도 입니다.
현재 비/눈/소나기가 오고 있지 않습니다.


In [8]:
weather_m = str(now) + '\n안산시 단원구 와동(57, 122)의 날씨 데이터입니다.\n기온은 ' + str(body['item'][3]['obsrValue']) + '도 입니다.\n' + weather_state

In [9]:
def send(a):
  pb = Pushbullet(access_token)
  device = pb.devices[1]

  push = pb.push_sms(device, a, weather_m)

In [ ]:
phoneNum=input('날씨 정보를 수신할 전화번호를 입력해주세요\n+8210과 함께  하이픈 없이 입력해주세요. \nex) 010-1234-5678 입력: +821012345678 >>')
send(phoneNum)